In [1]:
# /bin/bash
# source /opt/ros/noetic/setup.sh

In [2]:
! pip install cookiecutter rospkg numpy

In [1]:
import rospy
import numpy as np
from cookiecutter.main import cookiecutter
import collections
from pathlib import Path
import json
import shutil

In [2]:
def get_param(name, default):
    try:
        value = rospy.get_param(name, default)
    except ConnectionRefusedError:
        value = default
    print('param "{}" = "{}"'.format(name, value))
    return value

SEED = int(get_param('~seed', '178'))
WORLD_NAME = get_param('~world_name', 'apple_field')
MODEL_NAME_PREFIX = get_param('~model_name_prefix', 'apple')
OUT_PATH = Path(get_param('~out_path', Path.cwd() / '../generated/test01')).resolve()
MODEL_TEMPLATE = Path(get_param('~model_template', Path.cwd() / '../templates/apple_model')).resolve()
WORLD_TEMPLATE = Path(get_param('~world_template', Path.cwd() / '../templates/apple_world')).resolve()
ROW_COUNT = int(get_param('~row_count', '2'))
ROW_LENGTH = int(get_param('~row_length', '3'))
ROW_DIST = float(get_param('~row_dist', '4.0'))
CROP_DIST = float(get_param('~crop_dist', '2'))

shutil.rmtree(OUT_PATH, ignore_errors=True)
np.random.seed(SEED)


param "~seed" = "178"
param "~world_name" = "apple_field"
param "~model_name_prefix" = "apple"
param "~out_path" = "/home/facundo/catkin_ws/src/fields_ingnition/scripts/../generated/test01"
param "~model_template" = "/home/facundo/catkin_ws/src/fields_ingnition/scripts/../templates/apple_model"
param "~world_template" = "/home/facundo/catkin_ws/src/fields_ingnition/scripts/../templates/apple_world"
param "~row_count" = "2"
param "~row_length" = "3"
param "~row_dist" = "4.0"
param "~crop_dist" = "2"


In [3]:
# helper class to build the markers.json
class Markers:
    markers = []
    last_id = 0

    @staticmethod
    def next_id():
        Markers.last_id += 1
        return Markers.last_id

    @staticmethod
    def reset():
        Markers.markers = []
    
    @staticmethod
    def add_plant(x, y, z):
        id = Markers.next_id()
        Markers.markers.append({
            'marker_type': 'PLANT',
            'id': id,
            'translation': [x, y, z]
        })
        return id
    

    @staticmethod
    def add_fruit(x, y, z, plant_id):
        id = Markers.next_id()
        Markers.markers.append({
            'marker_type': 'FRUIT',
            'id': id,
            'translation': [x, y, z],
            'plant_id': plant_id
        })
        return id
    
    @staticmethod
    def dumps():
        return json.dumps(Markers.markers, indent=4)

In [4]:
models = {'list': []}

Markers.reset()

for x in range(ROW_COUNT):
    for y in range(ROW_LENGTH):
        model_name = 'apple_{}'.format(x * ROW_LENGTH + y)

        cookiecutter(str(MODEL_TEMPLATE),
             output_dir=str(OUT_PATH), 
             overwrite_if_exists=True, 
             no_input=True,
             extra_context={'world_name': WORLD_NAME, 'model_name': model_name})

        x_pos, y_pos, z_pos = (x) * ROW_DIST, (y) * CROP_DIST, 0
        models['list'].append({
            'model': model_name,
            'name': model_name,
            'pose': '{} {} 0 0 0 0'.format(x_pos, y_pos)
        })
        x_pos += np.random.uniform(-0.1, 0.1)
        y_pos += np.random.uniform(-0.1, 0.1)
        seed = np.random.randint(10000)
        dir = (OUT_PATH / WORLD_NAME / model_name).resolve()
        dir_blender = (Path.cwd() / '../blender').resolve()
        blend = str(dir_blender / 'apple_tree.blend')
        script = str(dir_blender / 'apple_gen.py')
        ! blender $blend --background --python $script -- --model_dir $dir -f one_leader_tree.json  --metadata --model
        plant_id = Markers.add_plant(x_pos, y_pos, z_pos)


cookiecutter(str(WORLD_TEMPLATE),
             output_dir=str(OUT_PATH), 
             overwrite_if_exists=True, 
             no_input=True,
             extra_context={'world_name': WORLD_NAME, 'models': models})

with open(OUT_PATH / WORLD_NAME / 'markers.json', 'w') as outfile:
    json.dump(Markers.markers, outfile, indent=4, sort_keys=True)

Blender 3.2.1 (hash a2d59b2dac9e built 2022-07-05 23:53:28)
Read prefs: /home/facundo/.config/blender/3.2/config/userpref.blend
Read blend: /home/facundo/catkin_ws/src/fields_ingnition/blender/apple_tree.blend
Namespace(model_dir='/home/facundo/catkin_ws/src/fields_ingnition/generated/test01/apple_field/apple_0', seed=1387, file='one_leader_tree.json', metadata=True, video=None, model=True)
Building  Horizontal Palmette
--- Generating leaders
----- Estructura generada, agregando hojas y frutas
/home/facundo/catkin_ws/src/fields_ingnition/blender/apple_gen.py:152: DeprecationWarning: Passing in context overrides is deprecated in favor of Context.temp_override(..), calling "object.join"
  bpy.ops.object.join({
/home/facundo/catkin_ws/src/fields_ingnition/blender/apple_gen.py:158: DeprecationWarning: Passing in context overrides is deprecated in favor of Context.temp_override(..), calling "object.join"
  bpy.ops.object.join({
/home/facundo/catkin_ws/src/fields_ingnition/blender/apple_gen.

apple_texture_2 <bpy_collection[1], Object.material_slots>
Branch1 <bpy_collection[1], Object.material_slots>
Leaf <bpy_collection[1], Object.material_slots>
apple_texture_4 <bpy_collection[1], Object.material_slots>
apple_texture_1 <bpy_collection[1], Object.material_slots>
Generando metadata
----Generando modelo
Collada export: Added image: apple_2.jpg.001.jpg
Collada export: Added image: apple_3.jpg.001.jpg
Collada export: Added image: apple.jpg.001.jpg
Collada export: Added image: brn1.png
Collada export: Added image: leaf_color.png.001.png.001.png
Collada export to: /home/facundo/catkin_ws/src/fields_ingnition/generated/test01/apple_field/apple_3/meshes/apple.dae
Info: Exported 5 Objects
Info: Exported 5 Objects

Blender quit
Error: Not freed memory blocks: 7, total unfreed memory 0.001244 MB
Blender 3.2.1 (hash a2d59b2dac9e built 2022-07-05 23:53:28)
Read prefs: /home/facundo/.config/blender/3.2/config/userpref.blend
Read blend: /home/facundo/catkin_ws/src/fields_ingnition/blende